# Show Me the City Data
> A tutorial for programatically reading Open Data from a Socrata database.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/cincy_skyline.jpg

![](../images/cincy_skyline.jpg)


## In this Post

Let's bring some first post energy!  I am excited to kick off this blog with a post about accessing Open Data in an easy, programmatic manner.

This notebook will demonstrate how to programatically read in Open Data from Socrata databases.  Socrata is a company commissioned by a large number of cities to host open data platforms.

These Open Data platforms are searchable and most importantly the data can be queried via an API.  These queries can be quite simple, or more complex.  We will dive into examples of both and be querying data in no time!

## Why is this Important

Querying and accessing data is a fundamental step of any data science workflow.  The mechanics of accessing data can get very messy, especially when it comes to government data.  Examples of this messiness include manually downloading files and saving to a user-defined location, appending multiple Excel files, scraping PDFs, and the list goes on.  Bottom line: it can get MESSY.

Luckily, the infrastructure provided by Socrata makes for a seamless experience that can be replicated by anyone with the Internet and Python.  You read that correctly, everything demonstrated in this post can be replicated with very little setup.  This is a huge perk for the sake of collaboration.  Additionally, in the event that an analysis needs to be re-run, perhaps on more recent data, the ability to query data programatically allows for minimal rework and minimal room for error.  Alright, enough hype let's get to it!

## Setup

Before we get rolling, it is important to be working in an environment with the necessary packages installed and available.  I like using Conda to manage environments (this sounds like a future blog post) and for the demonstration below it will be important to have pandas installed in your environment.

In [1]:
# load pandas
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

## Finding an API Endpoint

Before we dive into actually reading data, it is important to know where to look for potential datasets.  Most cities have their Open Data tied to the city's website.  In this case we are going to explore the City of Cincinnati's Open Data Portal, which can be found at this [link](https://data.cincinnati-oh.gov).

![](../images/2021-08-19-open-data-portal.png)


At this point, we can navigate into one of the categories or search for a specific dataset.  In this case, we select the 'Safety' category and will then choose the Police Calls dataset.

![](../images/2021-08-19-safety-data.png)

We are now able to Copy the API Endpoint by selecting the 'API' button and the 'Copy' button next to the endpoint.

![](../images/2021-08-19-PDI-endpoint.png)




## Reading in Data via API

You might notice that the API endpoint is just a link - try accessing the link below in your browser.  You will notice that the result is just a JSON string in your browser - this is the data we are pulling in imminently!

In order to read this into a pandas DataFrame, we utilize the pandas `read_json` function.  This function only has one required argument, `path_or_buf` and we set the value to our API endpoint.  More information on `read_json` can be found by looking at its docstring (simply run `?read_json`).

Note that in this initial read, we only pull back 1,000 total rows.  This is the default response when no other parameters are added to the endpoint URL.

In [ ]:
# define a variable to store our endpoint URL to be used throughout the post
crime_data_endpoint = 'https://data.cincinnati-oh.gov/resource/k59e-2pvf.json'

pd.read_json(path_or_buf = crime_data_endpoint)

## Socrata Query Language (SoQL)

In the above example, we simply query the entire dataset (albeit a sample of 1,000 rows) given we did not make any tweaks to the base URL.  However, we are able to arrive at fairly robust queries using the SoQL querying language. SoQL is a querying language developed by Socrata in order to make the process of requesting data via their API more efficient.  In their [documentation](https://dev.socrata.com/docs/queries/) they mention their intentions of mirroring the spirit of SQL and I believe they accomplished that goal quite well.

Being well versed in this querying language provides enormous benefits in the case of processing time.  It is far more efficient to execute data processing against the Socrata servers prior to bringing in data to your local machine.  For the remainder of this post we will explore examples of utilizing SoQL queries to refine our data before pulling it into our local machines.

### Example 1: Simple Filter

In our first example, we are going to execute a simple filter on our data by updating the `$WHERE` parameter.  As you would imagine, the default value for the `$WHERE` parameter is No Filter.

In our initial query above, we see that there is an `offense` column with a number of different values, stored as strings.  For this exercise, we are going to filter to instances where `offense` is equal to 'THEFT'.

In practice, we are able to execute this by modifying our API endpoint with the simple filter clause stamped on at the end.  To signal that we are going to add a SoQL clause to our endpoint, we first add a question mark (?) at the end of the initial endpoint, immediately following '.json':  `https://data.cincinnati-oh.gov/resource/k59e-2pvf.json?`

Now, we must add the actual where clause.  In this case, we first declare that we are updating the `$WHERE` parameter by adding `where=`.  We now add the filter statement, in this case `offense = 'THEFT'`.  This gives us a final query that looks like the following: 

`https://data.cincinnati-oh.gov/resource/k59e-2pvf.json?$WHERE=offense='THEFT'`.

> Note: from this point forward we will simply reference `crime_data_endpoint` when generating queries.  We will take advantage of f-String formatting that allows us to pass through a variable to a string statement (i.e. if `name` is a variable defined in the environment as the value of `'Joe'`, the statement `f"I am {name}"` will evaluate to `'I am Joe'`.

In [ ]:
#collapse-output
pd.read_json(f"{crime_data_endpoint}?$WHERE=offense='THEFT'")

With the query above, we see the data returned is exclusively limited to 'THEFT' offenses as desired.  We are now going to demonstrate the ability to string multiple `WHERE` clauses into the same query.

In order to do so, we tap into the logical operators allowed within the SoQL framework.  The following operators are available as illustrated in the [documentation](https://dev.socrata.com/docs/queries/where.html):
- `AND` (&)
- `OR` (|)
- `NOT`
- `IS NULL`
- `IS NOT NULL`

In this case, we are going to add the condition that an incident must have occurred in the `45202` zip code.  To accomplish this, we simply add the `&` symbol and an additional condition of `zip=45202`.

> Note: the SoQL documentation shows examples that include spaces in the endpoint URL.  The `pd.read_json()` function does not accept URLs with spaces.  Do your best to avoid using spaces, or replace the space with its URL encoding (in the case of a space that is `%20`).

In [ ]:
#collapse-output
pd.read_json(f"{crime_data_endpoint}?$WHERE=offense='THEFT'&zip=45202")

### Example 2: Date Filter

Working with dates can be finnicky in any scenario.  In this example, we aim to illustrate how to introduce a simple filter to limit the date range of our returned crime instances.

As you can see and infer from above, we have several fields that are represented as timestamps.  For this exercise, we are going to specifically focus on the `date_reported` field.  Let's explore how we would only look at records for which the `date_reported` value occurred within the last year.

For this exercise we will take advantage of the `datetime` library that assists with time and date operations.

In [ ]:
# import datetime library
import datetime as dt

# dynamically generate today's date
today = dt.date.today()

# dynamically arrive at the date exactly 365 days ago
year_ago_today = today - dt.timedelta(days = 365)

print(year_ago_today)

Now that we have our date for exactly one year ago, we can utilize it within our API call.  Similar to the simple filters above, we simply create a `$WHERE` clause, but in this case we look at instances where the `date_reported` field satisfies the condition of being after our `year_ago_today` value.

> Note: it is important to enclose the value for `year_ago_today` in quotes.  In this case I use single quotes for within the query, and enclose the entire query using double quotes.

In [ ]:
#collapse-output
pd.read_json(f"{crime_data_endpoint}?$WHERE=date_reported>='{year_ago_today}'")

### Example 3: Toggling Row Limit

By default, the row limit returned by the Socrata API is 1,000.  This is great for datasets smaller than 1,000 rows and illustrative blog posts, but for anything that actually matters it will be woefully insufficient.  Hence, we will now walk through the means for adjusting the row limit and integrating this into our queries.

This mechanic is really quite straightforward.  Similar to other SoQL query arguments, we simply define the parameter we are adjusting by adding `$LIMIT`, and then specifying the value for which to limit the row counts.  In the example below we limit this to `10`, however, you could ramp it up as large or small as needed.

> Note: I have not found an 'unlimited' value for this argument, thus if I know I would like to return all records I simply pass through an uncharacteristically large value that would not be anywhere near the max size of the data.  This is not a perfect solution but gets the job done when you know the rough size of your data.

In [ ]:
#collapse-output
pd.read_json(f"{crime_data_endpoint}?$LIMIT=10")

In [ ]:
#collapse-output
pd.read_json(f"{crime_data_endpoint}?$WHERE=offense='THEFT'&$LIMIT=5")

### Example 4: Select Subset of Columns

In this example we walkthrough the method for selecting only a subset of the columns in a dataframe.  This improves performance when you only need to work with a subset of columns by reducing the size of the dataframe.

The syntax is fairly straightforward and mirrors the methods for adding other parameters to our query.  Simply add a `$SELECT=` clause to the URL and list columns to include, separated by commas.

> Note: again, it is important to avoid spaces within the URL.  There is functionality to select columns under new names, like so: `$SELECT=suspect_gender AS gender`.  However, in this case we need to clean up the query to avoid any spaces in the URL by replacing them with the `%20` notation: `$SELECT=suspect_gender%20AS%20gender`.

In [ ]:
#collapse-output
pd.read_json(f"{crime_data_endpoint}?$SELECT=incident_no,date_reported,suspect_gender")

### Example 5: Putting it All Together with `$query`

Up until this point, we have walked through the parameters that can be modified within a query on an individual basis.  It was noted that multiple parameters can be combined by adding an `&` symbol between parameters, leading to more complex queries.  However, there is an easier way to string these parameters together, and it manifests itself within the `$query` parameter.

In these examples we will demonstrate how to string together a full-fledged query using the single `$query` parameter.  These queries will strongly resemble standard SQL queries that lack a `FROM` clause.  

To make things simpler, I have created a function (`generate_query`) to take care of some of the less readable pieces of these queries, notably the requirement to replace spaces (`' '`) with `'%20'`.  Furthermore, this function breaks the components of the query into chunks, making it very human readable.

In [ ]:
# function takes five arguments (endpoint_url and 4 components of the query) 
# function returns a full, cleaned up API call
def generate_query(endpoint_url, select_clause, where_clause, group_clause, order_by_clause):

    raw_query = (f"{endpoint_url}?$query="
                 f"{select_clause}%20"
                 f"{where_clause}%20"
                 f"{group_clause}%20"
                 f"{order_by_clause}"
                )
    
    final_query = str.replace(raw_query, " ", "%20")
    
    return final_query

Now, with our function in hand let's accomplish the following within two separate queries:
1. Select the columns `date_reported` and `offense` for records where `suspect_gender` is 'MALE'
2. Count the number of records by offense type for all crimes within the past year, order by the count in descending order

In [ ]:
## 1. Select the columns `date_reported` and `offense` for records where `suspect_gender` is 'MALE'

first_query = generate_query(endpoint_url = crime_data_endpoint,
               select_clause = "select date_reported, offense",
               where_clause = "where offense = 'THEFT'",
               group_clause = "",
               order_by_clause = "")

print(first_query)

In [ ]:
#collapse-output
pd.read_json(first_query)

In [ ]:
## 2. Count the number of records by offense type for all crimes within the past year, 
##    order by the count in descending order

second_query = generate_query(endpoint_url = crime_data_endpoint,
                              select_clause = "select offense, count(*) as n",
                              where_clause = f"where date_reported>='{year_ago_today}'",
                              group_clause = "group by offense",
                              order_by_clause = "order by count(*) desc"
                             )

print(second_query)

In [ ]:
#collapse-output
pd.read_json(second_query)

And with that we conclude the first post of the blog!  I hope you found this information helpful and I can't wait to deliver more content that explores the intersection of cities and data science.